## deltaLens Data Compare Sample

In [2]:
#imports

from delta_lens.deltaLens import DeltaLens, EntityComparer
from delta_lens.config import Config, Entity, Side, Transform, Defaults,ReferenceDataset, load_config
from delta_lens.sqliteExport import *
from delta_lens.csvExport import *
from datetime import date

In [3]:
#config

config = load_config("data/compare.config.json")


In [4]:
runName = f'tradeCompare_{date.today().strftime("%Y-%m-%d")}'

comparator = DeltaLens(runName,config)

comparator.execute(continue_on_error=True)



#### entity_compare_results
Table containing summary of each entity comparison

In [5]:
comparator.con.execute("SELECT * FROM  entity_compare_results").fetch_df()

,entity,rows_left,rows_right,rows_fully_matched,error_text,success
0,trade,42999117,43000000,42113850,None,1


#### [entitiy]_compare_field_summary
Entity specific table containing summary of field comparison

In [6]:
comparator.con.execute("SELECT * FROM trade_compare_field_summary").fetch_df()

,field,total,matches,match_percentage
0,trade_type,42999117,42999117.0,100.000000
1,price,42999117,42113850.0,97.940002
2,quantity,42999117,42999117.0,100.000000
3,timestamp,42999117,42999117.0,100.000000
4,symbol,42999117,42999117.0,100.000000


#### [entity]_compare
Entity specific table containing details of comparison

In [7]:
comparator.con.execute(" WITH CTE as (SELECT * FROM  trade_compare WHERE _full_match = 0) SELECT * FROM CTE USING SAMPLE 5;").fetch_df()

,trade_id,trade_type_left,trade_type_right,trade_type_match,price_left,price_right,price_match,quantity_left,quantity_right,quantity_match,timestamp_left,timestamp_right,timestamp_match,symbol_left,symbol_right,symbol_match,_exists_left,_exists_right,_full_match
0,4921007,BUY,BUY,True,933.90,122.18,False,4587,4587,True,2024-09-04 20:34:38,2024-09-04 20:34:38,True,TSLA,TSLA,True,True,True,False
1,13555067,SELL,SELL,True,323.85,610.87,False,2536,2536,True,2024-07-14 20:35:42,2024-07-14 20:35:42,True,AAPL,AAPL,True,True,True,False
2,24345859,SELL,SELL,True,161.84,861.80,False,7060,7060,True,2025-01-08 20:37:01,2025-01-08 20:37:01,True,NVDA,NVDA,True,True,True,False
3,1483911,SELL,SELL,True,66.11,439.30,False,9570,9570,True,2024-08-18 20:34:13,2024-08-18 20:34:13,True,WMT,WMT,True,True,True,False
4,5180774,SELL,SELL,True,758.10,147.89,False,2459,2459,True,2025-02-05 20:34:40,2025-02-05 20:34:40,True,TSLA,TSLA,True,True,True,False


#### Export to sqlite
Function to export above tables to on-disk format. Sqlite is a popular single file database, well supported in many languages,  for which lots of query tools exist. See [Datasette](https://datasette.io/) package.

In [ ]:
export_to_sqlite(comparator.con, f'{runName}.sqlite')

'tradeCompare_2025-02-15.sqlite'

In [9]:
export_to_csv_archive(comparator.con,f'{runName}.tar.gz' )

'tradeCompare_2025-02-15.tar.gz'